# Maximizing accuracy under fairness constraints (C-SVM and C-LR)

In [144]:
from __future__ import division

import pandas as pd
import numpy as np

# import urllib2
import os,sys
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn import feature_extraction
from sklearn import preprocessing
from random import seed, shuffle
import a2_loss_funcs as lf 
import a2_utils as ut



In [145]:
df = pd.read_csv("../data/compas-scores-two-years.csv")
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [146]:
df.shape

(7214, 53)

## Preprocessing

Selecting the features we will use for analysis, the target variable and the sensitive attributes

In [147]:
FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "priors_count", "c_charge_degree", 'score_text', 'decile_score']
CONT_VARIABLES = ["priors_count", "decile_score"] 
CLASS_FEATURE = "two_year_recid" 
SENSITIVE_ATTRS = ["race"]

In [148]:
for column in FEATURES_CLASSIFICATION:
    print(column)
    print(df[column].unique())
    print("--------------------------")

age_cat
['Greater than 45' '25 - 45' 'Less than 25']
--------------------------
race
['Other' 'African-American' 'Caucasian' 'Hispanic' 'Native American'
 'Asian']
--------------------------
sex
['Male' 'Female']
--------------------------
priors_count
[ 0  4  1  2 14  3  7  6  5 13  8  9 21 20 15 10 12 28 19 11 22 23 25 24
 36 18 16 33 17 30 27 38 26 37 29 35 31]
--------------------------
c_charge_degree
['F' 'M']
--------------------------
score_text
['Low' 'High' 'Medium']
--------------------------
decile_score
[ 1  3  4  8  6 10  5  9  2  7]
--------------------------


In [149]:
df = df.dropna(subset=["days_b_screening_arrest"]) 
data = df.to_dict('list')
for k in data.keys():
    data[k] = np.array(data[k])


The following preprocessing steps are taken based on the [propublica GitHub page](https://github.com/propublica/compas-analysis)


- If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense.
- We coded the recidivist flag -- is_recid -- to be -1 if we could not find a compas case at all.
- In a similar vein, ordinary traffic offenses -- those with a c_charge_degree of 'O' -- will not result in Jail time are removed (only two of them).
- We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.

In [150]:
idx = np.logical_and(data["days_b_screening_arrest"]<=30, data["days_b_screening_arrest"]>=-30)

idx = np.logical_and(idx, data["is_recid"] != -1)

idx = np.logical_and(idx, data["c_charge_degree"] != "O") 

idx = np.logical_and(idx, data["score_text"] != "NA")

idx = np.logical_and(idx, np.logical_or(data["race"] == "African-American", data["race"] == "Caucasian"))

for k in data.keys():
    data[k] = data[k][idx]

In [156]:
unique_values, value_counts = np.unique(data['race'], return_counts=True)

# Display unique values and their counts
for value, count in zip(unique_values, value_counts):
    print(f"{value}: {count}")

African-American: 3175
Caucasian: 2103


In [157]:
y = data[CLASS_FEATURE]
y[y==0] = -1

print("Number of people recidivating within two years")
print(pd.Series(y).value_counts())

Number of people recidivating within two years
-1    2795
 1    2483
Name: count, dtype: int64


In [128]:
X = np.array([]).reshape(len(y), 0) 
x_control = defaultdict(list)

Convert the discrete variables to one-hot encoded columns and scale the continuous variables

In [129]:
feature_names = []
for attr in FEATURES_CLASSIFICATION:
    vals = data[attr]
    if attr in CONT_VARIABLES:
        vals = [float(v) for v in vals]
        vals = preprocessing.scale(vals) 
        vals = np.reshape(vals, (len(y), -1)) 

    else: 
        lb = preprocessing.LabelBinarizer()
        lb.fit(vals)
        vals = lb.transform(vals)

    # add to sensitive features dict
    if attr in SENSITIVE_ATTRS:
        x_control[attr] = vals


    # add to learnable features
    X = np.hstack((X, vals))

    if attr in CONT_VARIABLES: # continuous feature, just append the name
        feature_names.append(attr)
    else: # categorical features
        if vals.shape[1] == 1: # binary features that passed through lib binarizer
            feature_names.append(attr)
        else:
            for k in lb.classes_: # non-binary categorical features, need to add the names for each cat
                feature_names.append(attr + "_" + str(k))

In [130]:
# convert the sensitive feature to 1-d array
x_control = dict(x_control)
for k in x_control.keys():
    assert(x_control[k].shape[1] == 1) # make sure that the sensitive feature is binary after one hot encoding
    x_control[k] = np.array(x_control[k]).flatten()

In [132]:
perm = list(range(0, X.shape[0]))
shuffle(perm)
X = X[perm]
y = y[perm]
for k in x_control.keys():
    x_control[k] = x_control[k][perm]


In [133]:
X = ut.add_intercept(X)

feature_names = ["intercept"] + feature_names
assert(len(feature_names) == X.shape[1])
print ("Features we will be using for classification are:", feature_names, "\n")


Features we will be using for classification are: ['intercept', 'age_cat_25 - 45', 'age_cat_Greater than 45', 'age_cat_Less than 25', 'race', 'sex', 'priors_count', 'c_charge_degree', 'score_text_High', 'score_text_Low', 'score_text_Medium', 'decile_score'] 



Compute the p-rule in the original data

In [134]:
ut.compute_p_rule(x_control["race"], y) 


Total data points: 5278
# non-protected examples: 2103
# protected examples: 3175
Non-protected in positive class: 822 (39%)
Protected in positive class: 1661 (52%)
P-rule is: 134%


133.84228978676936

Split the data into train and test


In [135]:
train_fold_size = 0.7
x_train, y_train, x_control_train, x_test, y_test, x_control_test = ut.split_into_train_test(X, y, x_control, train_fold_size)


## Logictic Regression

Normal Logistic Regression without applying fairness constraints. For this the the "apply_fairness_constraints" value should be set to 0.

In [136]:
apply_fairness_constraints = 0
apply_accuracy_constraint = 0
sep_constraint = 0

loss_function = lf._logistic_loss
sensitive_attrs = ["race"]
sensitive_attrs_to_cov_thresh = {}
gamma = None

In [137]:
w = ut.train_model(x_train, y_train, x_control_train, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma)
train_score, test_score, correct_answers_train, correct_answers_test = ut.check_accuracy(w, x_train, y_train, x_test, y_test, None, None)
distances_boundary_test = (np.dot(x_test, w)).tolist()
all_class_labels_assigned_test = np.sign(distances_boundary_test)
correlation_dict_test = ut.get_correlations(None, None, all_class_labels_assigned_test, x_control_test, sensitive_attrs)
cov_dict_test = ut.print_covariance_sensitive_attrs(None, x_test, distances_boundary_test, x_control_test, sensitive_attrs)
p_rule = ut.print_classifier_fairness_stats([test_score], [correlation_dict_test], [cov_dict_test], sensitive_attrs[0])	


Accuracy: 0.70
Protected/non-protected in +ve class: 53% / 29%
P-rule achieved: 182%
Covariance between sensitive feature and decision from distance boundary : 0.120


Logistic Regression with fairness constraints. For this the the "apply_fairness_constraints" value should be set to 1 and the "sensitive_attrs_to_cov_thresh" should pass the dict of fairness column.

In [138]:
apply_fairness_constraints = 1
apply_accuracy_constraint = 0
sep_constraint = 0

loss_function = lf._logistic_loss
sensitive_attrs = ["race"]
sensitive_attrs_to_cov_thresh = {"race":0}
gamma = None

In [139]:
w = ut.train_model(x_train, y_train, x_control_train, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma)
train_score, test_score, correct_answers_train, correct_answers_test = ut.check_accuracy(w, x_train, y_train, x_test, y_test, None, None)
distances_boundary_test = (np.dot(x_test, w)).tolist()
all_class_labels_assigned_test = np.sign(distances_boundary_test)
correlation_dict_test = ut.get_correlations(None, None, all_class_labels_assigned_test, x_control_test, sensitive_attrs)
cov_dict_test = ut.print_covariance_sensitive_attrs(None, x_test, distances_boundary_test, x_control_test, sensitive_attrs)
p_rule = ut.print_classifier_fairness_stats([test_score], [correlation_dict_test], [cov_dict_test], sensitive_attrs[0])	


Accuracy: 0.68
Protected/non-protected in +ve class: 42% / 42%
P-rule achieved: 99%
Covariance between sensitive feature and decision from distance boundary : 0.025


## Support Vector Machine

Without fairness

In [140]:
apply_fairness_constraints = 0
apply_accuracy_constraint = 0
sep_constraint = 0

loss_function = lf._hinge_loss
sensitive_attrs = ["race"]
sensitive_attrs_to_cov_thresh = {}
gamma = None

In [141]:
w = ut.train_model(x_train, y_train, x_control_train, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma)
train_score, test_score, correct_answers_train, correct_answers_test = ut.check_accuracy(w, x_train, y_train, x_test, y_test, None, None)
distances_boundary_test = (np.dot(x_test, w)).tolist()
all_class_labels_assigned_test = np.sign(distances_boundary_test)
correlation_dict_test = ut.get_correlations(None, None, all_class_labels_assigned_test, x_control_test, sensitive_attrs)
cov_dict_test = ut.print_covariance_sensitive_attrs(None, x_test, distances_boundary_test, x_control_test, sensitive_attrs)
p_rule = ut.print_classifier_fairness_stats([test_score], [correlation_dict_test], [cov_dict_test], sensitive_attrs[0])	


Accuracy: 0.68
Protected/non-protected in +ve class: 57% / 37%
P-rule achieved: 155%
Covariance between sensitive feature and decision from distance boundary : 0.096


With fairness

In [142]:
apply_fairness_constraints = 1
apply_accuracy_constraint = 0
sep_constraint = 0

loss_function = lf._hinge_loss
sensitive_attrs = ["race"]
sensitive_attrs_to_cov_thresh = {"race":0}
gamma = None

In [143]:
w = ut.train_model(x_train, y_train, x_control_train, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma)
train_score, test_score, correct_answers_train, correct_answers_test = ut.check_accuracy(w, x_train, y_train, x_test, y_test, None, None)
distances_boundary_test = (np.dot(x_test, w)).tolist()
all_class_labels_assigned_test = np.sign(distances_boundary_test)
correlation_dict_test = ut.get_correlations(None, None, all_class_labels_assigned_test, x_control_test, sensitive_attrs)
cov_dict_test = ut.print_covariance_sensitive_attrs(None, x_test, distances_boundary_test, x_control_test, sensitive_attrs)
p_rule = ut.print_classifier_fairness_stats([test_score], [correlation_dict_test], [cov_dict_test], sensitive_attrs[0])	


Accuracy: 0.69
Protected/non-protected in +ve class: 51% / 40%
P-rule achieved: 128%
Covariance between sensitive feature and decision from distance boundary : 0.030
